In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
total_data=pd.read_csv("../data/total_data.csv",index_col=0)
total_data_demographic=pd.read_csv("../data/total_data_demographic.csv",index_col=0)

In [ ]:
total_data[['customer_id','PRODUCT_ID','DAY']].groupby(['customer_id','PRODUCT_ID']).count().sort_values(by='DAY')

In [ ]:
a=total_data[['customer_id','DAY','PRODUCT_ID','WEEK_NO']]
a.customer_id=a.customer_id.transform(lambda x: str(x))

In [ ]:
encoded = pd.get_dummies(a.customer_id,prefix='bought_by')
b=pd.concat([a,encoded],axis=1)

In [ ]:
# b['last_purchase_day']=b.groupby(['customer_id','PRODUCT_ID'])['DAY'].shift()

In [ ]:
b.drop('customer_id',inplace=True,axis=1)

In [ ]:
b=b.groupby(['DAY','PRODUCT_ID','WEEK_NO'],as_index=False).sum()

In [ ]:
product_data=pd.read_csv("../dataset/product.csv")

In [ ]:
total_data=pd.merge(product_data[['PRODUCT_ID','DEPARTMENT','BRAND','COMMODITY_DESC','SUB_COMMODITY_DESC']],b,on='PRODUCT_ID')

In [ ]:
total_data

In [ ]:
future=product_data.copy()
filtered_product_ids=list(set(total_data.PRODUCT_ID))

In [ ]:
future=future[future['PRODUCT_ID'].isin(filtered_product_ids)]
future=future[['PRODUCT_ID', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC','SUB_COMMODITY_DESC']]
future['DAY']=713
future['WEEK_NO']=103

In [ ]:
for i in range(6):
    future_day=future.copy()
    future_day['DAY']=i+714
    future=pd.concat([future,future_day])#,on=['PRODUCT_ID',	'DEPARTMENT',	'BRAND',	'COMMODITY_DESC','SUB_COMMODITY_DESC'])

In [ ]:
future=future.reindex(columns=total_data.columns.tolist(),fill_value=0)

In [ ]:
future

In [ ]:
def train_test_split(X,y,test_size=0.25,val_size=.2):
    y=y.astype('float64')
    X_test_Split=int((X.shape[0])*(test_size+val_size))
    X_val_Split=int((X.shape[0])*val_size)
    y_test_Split=int((y.shape[0])*(test_size+val_size))
    y_val_Split=int((y.shape[0])*val_size)
    print(X_test_Split,X_val_Split,y_test_Split,y_val_Split)
    X_Train=X[: -X_test_Split]
    X_Test=X[-X_test_Split:-X_val_Split]
    X_Val=X[-X_val_Split:]
    y_Train=y[: -y_test_Split]
    y_Test=y[-y_test_Split:-y_val_Split]
    y_Val=y[-y_val_Split:]
    return [X_Train, X_Test,X_Val, y_Train, y_Test,y_Val]

In [ ]:
#one hot encoding categorical data for modelling
encoded = pd.get_dummies(total_data[total_data.columns[total_data.dtypes==object]])
total_data_encoded = pd.concat([encoded, total_data[total_data.columns[total_data.dtypes != object]]],axis=1)

X = total_data_encoded.drop('bought_by_2337',axis=1)
y = total_data_encoded['bought_by_2337']
future=future.drop('bought_by_2337',axis=1).reset_index()
[X_Train, X_Test,X_Val, y_Train, y_Test,y_Val] = train_test_split(X,y,test_size=0.2,val_size=.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor

#XGBoost
xgb_mdl = XGBRegressor(nthread=-1).fit(X_Train.values,y_Train.values)
xgb_mdl

In [ ]:
encoded = pd.get_dummies(future[future.columns[future.dtypes==object]])
future_encoded = pd.concat([encoded, future[future.columns[future.dtypes != object]]],axis=1)

In [ ]:
future_pred = xgb_mdl.predict(future_encoded.values)

In [ ]:
future_pred

In [ ]:
future=future[['PRODUCT_ID'	,'DEPARTMENT'	,'BRAND'	,'COMMODITY_DESC'	,'SUB_COMMODITY_DESC',	'DAY',	'WEEK_NO']]

In [ ]:
future['PREDICTION']=future_pred

In [ ]:
temp=future.groupby('DAY')

In [ ]:
day_suggestions=future.sort_values(['DAY','PREDICTION'],ascending=False).groupby(['DAY'	,'WEEK_NO']).head(5).sort_values(['PRODUCT_ID','DAY'])

In [ ]:
day_suggestions[day_suggestions['DAY']==713]

In [ ]:
# y_pred_train = xgb_mdl.predict(X_Train.values)
# y_pred = xgb_mdl.predict(X_Test.values)

In [ ]:
# ans=y_pred
# temp=[0 if(x<.02) else 1 for x in (ans*100).astype(int)/100 ]

In [ ]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# a = np.array(y_Test).reshape(1,-1)
# b = np.array(temp).reshape(1,-1)

# cosine_similarity(a,b)

In [ ]:
# from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, average_precision_score

# print("Train Data Classification Report:\n")
# print(classification_report(y_Train,y_pred_train))

# print("Test Data Classification Report:\n")
# print(classification_report(y_Test,y_pred))

# cm = pd.DataFrame(confusion_matrix(y_Test, y_pred))#, index = list(set(y)), columns = list(set(y)))


In [ ]:
# cm = pd.DataFrame(confusion_matrix(y_Test, y_pred))#, index = list(set(y)), columns = list(set(y)))
# cm

In [ ]:
future[(future.PRODUCT_ID in filtered_product_ids).item()]

In [ ]:
filtered_product_ids=list(set(total_data.PRODUCT_ID))